### Agent
AI Agent 算是 AI 界当前最热门的话题之一，其实这个概念应该在很早之前就已经有了，但是至今也没有一个完整的定义究竟什么才能算一个Agent。所以这里引用下维基百科的定义
> 智能代理（英语：intelligent agent），或译为智能主体。在人工智能领域，智能代理指一个可以观察周遭环境并作出行动以达致目标的自主实体。它通常是指（但不一定是）一个软件程序。“智能代理”是目前人工智能研究的一个核心概念，统御和联系着各个子领域的研究。

在维基的定义中，写到Agent可以观察周遭环境，这个其实有点虚，因为对于人来说观察都是通过去看，去听进行观察的，而对于NLP模型来说，肯定是没有这样的能力的。所以我这里不给Agent下定义，直接来一起反观一下，之前我们写的各种应用究竟解决了啥，还有啥不能解决。

最初我们知道LLM，它可以看成一个输入是字符串输出是字符串的黑盒(不考虑多模态的情况)。在输入端，我们通过模板字符串实现了Prompt的复用。在输出端，我们通过各种OutputParser实现了输出格式的控制。然后我们通过langchain提供的RAG，实现了外挂知识，让模型在不微调的情况下也可以拥有领域知识。理想状态下，只要有对应的知识提供，模型可以回答你任何问题了。但是我们依然不能让模型完成以下的事情：
* 让模型告诉我下周某个地方的天气，并根据天气以及可能的交通状况推荐旅游规划。
* 让模型帮我写一个完全可靠的代码
* 让模型帮我做一道回锅肉？(好像有点离谱了)
以上这些要求和之前的需求不一样的在于，不止需要模型能思考，能给出答案，还需要模型能通过使用工具帮你完成任务。你可以稍微险隘得理解Agent和普通LLM应用在于能不能"使用工具"。

所以我们先来聊聊所谓的*工具*。对于程序抽象来说，工具其实无非就是一次函数调用，它可能是通过一次restful或者rpc这种形式调用的，也可能是通过子进程的方式调用的。它也可以是任何东西，比如查询下周某个地方的天气，比如将下载好的某个图片保存到你磁盘的某个位置，凡此种种都可以是一个工具。比如回到我们的第一个问题，一个模型要根据某个地方下周的天气情况帮我们规划推荐旅游的地方，它是肯定不知道下周那个地方天气情况如何的。所以这时候你需要提供给它一个接口，通过这个接口它可以查询到下周是什么天气，从这个接口拿到天气结果后，它会继续分析，这个天气的接口在langchain中就是一个Tool。有了这个接口之后，模型会拿到天气情况，然后继续询问之前的问题，在这个过程中，我们能注意到，这个是一个多轮对话，因此在多轮对话的情形下，每一轮对话的时候模型怎么知道之前对话的内容是啥，也就是它如何能记住之前我们聊了啥，这就是Memory组件需要完成的。在有了Tool和Memory之后，我们还要思考，如何让LLM去调用Tool。如果是我们通过程序去调用交给它，那其实决策者本质还是我们，不是模型。因此我们要引导模型让它去决策什么时候应该调用哪个Tool，是否需要调用这个Tool，调用这个Tool之后得到的结果又如何处理，这就是Agent组件的作用。

之前有人总结，作为一个Agent，需要有规划，记忆，行动和工具这四个方面的能力。而Tool，Memory和Agent(其中Agent包括Agent和AgentExcutor)就分别对应了工具，记忆，规划和行动这四个方面。

#### Tools
对于Agent来说，需要完成的任务肯定是各式各样的，因此langchain提供了大量的可以直接使用的Tool。但是和其他组件不同，即便如此，也很容易出现自己想要的Tool没有提供的情况，因此自定义Tool的需求应该是非常常见，我们这里介绍一两个langchain给定的Tool，然后尝试去自定义一个Tool。
首先比如我要去查询一下下周某个地方的天气，我们可以通过某些专业的天气平台提供的api去获取，也可以直接通过搜索引擎去查询。一般来说通过专门的天气平台获取的数据干扰比较小，通过搜索引擎获取的数据可能干扰比较大，但是搜索引擎可以查询的数据是任意的(虽然可能结果并没啥营养)。
##### SearchAPI
我们经常需要拿着一个问题去问搜索引擎，因此搜索引擎是一个比较重要和常用的Tool，langchain也内置了不少的搜索引擎，但是可惜很多因为各种原因不能直接访问。SearchAPI是其中较少的可以直接访问到的

In [3]:
from langchain_community.utilities import SearchApiAPIWrapper
import os
import getpass

if os.environ.get('SEARCHAPI_API_KEY') is None:
    os.environ['SEARCHAPI_API_KEY'] = getpass.getpass()

search = SearchApiAPIWrapper()
search.run('下周成都天气如何')

'25日（今天）. 多云转阵雨. 32/21℃. <3级 · 26日（明天）. 阵雨转多云. 22/19℃. <3级 · 27日（后天）. 阴转阵雨. 27/19℃. <3级 · 28日（周五）. 阴. 28/19℃. <3级 · 29日（周六 ...\n周二（2日） 晴 35℃/23℃ 东南风转东北风 <3级 · 周三（3日） 多云转雨 34℃/24℃ 西北风转东北风 <3级 · 周四（4日） 雨 33℃/25℃ 东北风转北风 <3级 · 周五（5日） 雨 35℃/24℃ ...\n今天(06-25). 多云转阵雨. 21/32℃ ; 明天(06-26). 阵雨转多云. 19/22℃ ; 周四(06-27). 阴转阵雨. 19/27℃ ; 周五(06-28). 阴. 19/28℃ ; 周六(06-29). 多云. 21/32℃.\n成都未来15天天气预报 ; 星期六. 2024-06-29. 多云. 21℃ ～ 32℃. 微风<3级 ; 星期日. 2024-06-30. 多云转阵雨. 22℃ ～ 33℃. 微风<3级 ; 星期一. 2024-07-01. 多云. 23℃ ～ 35℃.\n30日（周日）. 多云转阵雨. 33/22℃. <3级. 1日（周一）.\n未来30天将有25天下雨，最高温35°（07月01日，07月02日，07月05日），最低温19°（06月26日，06月27日，06月28日）。\n成都往日天气预报. 6月23日 周日 多云 24°/33°; 6月22日 周六 小雨 24°/34°; 6月21日 周五 小雨 23°/31°; 6月20日 周四 多云转中雨 23°/30°; 6月19日 周三 小雨转晴 ...\n周三. 6/26. 72° 66°. 幾場陣雨. 大致多雲. 60%. 周四. 6/27. 81° 66°. 多雲. 零星有雨和細雨. 25%. 周五. 6/28. 82° 66°. 多雲. 多雲. 25%. 周六. 6/29. 90° 70°. 大致 ...\n今天(05-23). 多云转晴. 21/34℃ ; 明天(05-24). 多云转中雨. 23/34℃ ; 周六(05-25). 小雨转阴. 20/31℃ ; 周日(05-26). 多云. 21/31℃ ; 周一(05-27). 多云转小雨. 20/30℃.\n成都24小时天气预

#### Python REPL
我们经常希望模型能用代码帮我们解决一个问题，但是实际情况大部分都是模型会给你一个可能没啥大问题，但是很多小问题的代码。如果我们希望模型在给我们代码之前，先把代码运行下，如果没有语法错误再给我们，如果有错误就继续重新修改直到没有错误。这时候就需要一个运行环境。Python的REPL就是这么一个Tool。

In [7]:
from langchain_community.utilities import PythonREPL

python_repl = PythonREPL()
python_repl.run('print([x for x in range(10)])')

'[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]\n'

可以看出来，以上工具其实都是函数。需要包装为Tool才能被Agent自动调用。因为Agent需要自己思考什么时候调用哪个函数，那么其实你就需要将函数的描述给到Agent

In [ ]:
from langchain.agents import Tool, AgentExecutor
from langchain_community.utilities import SearchApiAPIWrapper
import os
import getpass

if os.environ.get('SEARCHAPI_API_KEY') is None:
    os.environ['SEARCHAPI_API_KEY'] = getpass.getpass()

search = SearchApiAPIWrapper()

search_tool = Tool(
    name="search_tool",
    func=search.run,
    description="你可以用它调用搜索引擎进行询问",
)

可以看出，其实要自定义一个Tool还是很简单的，你只需要提供对应的函数，工具名称和描述即可。langchain除了提供了很多的[Tool](https://python.langchain.com/v0.1/docs/integrations/tools/)可以使用之外，还将功能比较类似的一些Tool抽象成了[ToolKit](https://python.langchain.com/v0.1/docs/integrations/toolkits/)

#### Agent
有了Tool之后，我们可以编排Agent了。首先来说，Agent的实现有很多种，在langchain中也实现了好几种[Agent](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)。其中比较比较出名的是ToolCalling和ReAct。前者支持的特性比较多，比如多输入Tool，并行的函数调用等，但是有个很大的要求，就是模型本身必须能有函数调用能力。ReAct的话不支持多输入Tool，也不能并行函数调用，但是可以支持所有函数。为了通用性，我们这里以ReAct这种Agent作为例子，对其他Agent类型感兴趣可以去上面链接看文档。

##### ReAct
ReAct沦为发表于2022年底，当时还没有ChatGPT。论文主要阐述了一种结合推理(Reson)和行动(Acting)的语言模型应用范式。旨在通过交替生成推理轨迹和特定任务的动作，使得大型语言模型（LLMs）在解决多样化的语言推理和决策任务时能够实现更深层次的协同作用。

<center>
  <img src="images/ReAct.png">
</center>

从ReAct的提示词来看，它会引导模型进行多次思考，每次思考都会去考虑能不能得到最终结论。如果不能就会从给定的Tool里面选择一个合适的去完成一个Action。当这个Action完成后，它又会思考目前能不能得出最终结论，如果不能，它会继续进行上面的循环。文字其实读起来有点抽象，我们写一个ReAct的Agent来看看吧

In [1]:
from langchain.agents import create_react_agent, AgentExecutor, load_tools, Tool
from langchain import hub
from langchain_community.utilities import SearchApiAPIWrapper
from libs.llm.qwen import qwen
import os
import getpass

if os.environ.get('SEARCHAPI_API_KEY') is None:
    os.environ['SEARCHAPI_API_KEY'] = getpass.getpass()

search = SearchApiAPIWrapper()
search_tool = Tool(
    name="search_tool",
    func=search.run,
    description="你可以用它调用搜索引擎进行询问",
)

tools = load_tools(['llm-math', 'searchapi'], llm=qwen)

agent = create_react_agent(llm=qwen, tools=tools + tools, prompt=hub.pull("hwchase17/react"))
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
# agent_executor.invoke({"input": "中，日，韩2000年生育率的平均值是多少？"})
agent_executor.invoke({"input": "根据成都下周天气推荐一个旅游的攻略"})



> Entering new AgentExecutor chain...
 我需要了解成都下周的天气情况，以便为旅游攻略提供建议。
Action: searchapi
Action Input: "成都下周天气"
成都天气预报，及时准确发布中央气象台天气信息，便捷查询成都今日天气， ... 北方高温下周初再度发力江南华南强 ...
今天(06-25). 多云转阵雨. 21/32℃ ; 明天(06-26). 阵雨转多云. 19/22℃ ; 周四(06-27). 阴转阵雨. 19/27℃ ; 周五(06-28). 阴. 19/28℃ ; 周六(06-29). 多云. 21/32℃.
成都未来15天天气预报 ; 星期六. 2024-06-29. 多云. 21℃ ～ 32℃. 微风<3级 ; 星期日. 2024-06-30. 多云转阵雨. 22℃ ～ 33℃. 微风<3级 ; 星期一. 2024-07-01. 多云. 23℃ ～ 35℃.
18℃. 多云. 无持续风向. 微风. 06/27 周四. 阴. 无持续风向. 微风. 27℃. 20℃. 阵雨. 无持续风向. 微风. 06/28 周五. 阵雨. 无持续风向. 微风. 27℃. 21℃. 阴. 无持续风向.
未来30天将有25天下雨，最高温35°（07月01日，07月02日，07月05日），最低温19°（06月26日，06月27日，06月28日）。
24日（今天）. 阴转多云. 32/21℃. <3级 · 25日（明天）. 多云转阵雨. 30/21℃. <3级 · 26日（后天）. 阵雨转多云. 26/19℃. <3级 · 27日（周四）. 小雨. 24/20℃. <3级 · 28日（周 ...
成都一周天气预报 ; 06-11(今天). 晴. 18～30℃ ; 06-12(明天). 多云. 20～32℃ ; 06-13(周四). 晴. 20～30℃ ; 06-14(周五). 晴. 22～35℃ ; 06-15(周六). 多云. 22～38℃. 下周成都的天气预报显示，大部分时间会有阵雨或小雨，气温在19℃至33℃之间波动。虽然有雨，但温度适宜旅行。
Final Answer: 推荐成都旅游攻略如下：

1. 行程安排：由于下周成都雨水较多，建议携带雨具并合理规划室内活动和室外活动的比例。可以安排一些适合阴雨天的

{'input': '根据成都下周天气推荐一个旅游的攻略',
 'output': '推荐成都旅游攻略如下：\n\n1. 行程安排：由于下周成都雨水较多，建议携带雨具并合理规划室内活动和室外活动的比例。可以安排一些适合阴雨天的行程，如参观博物馆、艺术馆、大熊猫繁育研究基地等室内景点。\n\n2. 穿着建议：根据天气预报，气温适中，可准备薄外套和短袖衣物，并随时带上雨伞或雨衣。\n\n3. 室外活动：选择雨势较小或者雨后的日子游览成都市区的文化名胜，如武侯祠、锦里古街、宽窄巷子等，同时可以体验成都特色的茶馆文化。\n\n4. 自然景观：鉴于可能的降雨，可以选择在雨后去青城山或峨眉山等自然景区欣赏云雾缭绕的山水美景。\n\n总之，在制定旅游计划时需关注实时天气预报，灵活调整行程安排，确保旅行体验。'}